In [1]:
from tqdm.notebook import trange
import pathlib
import matplotlib.pyplot as plt
import numpy as np

In [2]:
rng = np.random.default_rng() #Random number generator that outputs uniformly distributed floats in the range [0,1)

<h1>Assignment 2:</h1>

<h2>Calculating the parameters:</h2>

The Hamiltonian is given by $$H = -J \sum_{\left\langle ij \right\rangle}{s_i s_j}$$ where $s_i \in \{+1,-1\}$.

We need to find a suitable value of $J$ such that the critical temperature $T_c$ as given by the mean field theory approximation is $2$. The relation between $J$ and $T_c$ is given as $JC\beta_c = \frac{JC}{kT_c}=1$ where $C=4$ for the 2D Ising model. We will define, for now, that $k=2$ and correct our obtained values accordingly later.

This conveniently gives us $J=1$. Also, $\beta_c = \frac{1}{kT_c} = \frac{1}{4}$.

We will consider periodic boundary conditions.

<h2>Monte-Carlo step:</h2>

The spin-up state is taken as $+1$ and the spin-down state is taken as $-1$.

In one Monte-Carlo step, we flip the state of one spin and calculate the resulting energy difference $\Delta E = E_f - E_i$. When a spin is flipped, only $4$ terms are affected in the Hamiltonian, and they are the interactions between the nearest neighbour spins of the flipped spin and the flipped spin itself. So, instead of recalculating the entire Hamiltonian every time, we can simply calculate the energy difference from these $4$ terms.

Let us say spin $s_{(i,j)}$ was flipped (which is the spin at row $i$ and column $j$). The energy difference is then $$\Delta E = 2Js_{(i,j)} \cdot \left( s_{(i+1,j)} + s_{(i,j+1)} + s_{(i-1,j)} + s_{(i,j-1)} \right)$$ since we just flipped the sign of these $4$ terms. Note that the indices are considered modulo $N$ because of periodic boundary conditions.

Then we can test if the new state satisfies our conditions and if it does, we'll flip the spin.

<h2>Speeding up the simulation:</h2>

If we consider the lattices for fixed $N$, in each timestep we are performing basically the same operation on all of them, and each step follows the same sequence of spins. So instead of evolving them one by one, we can process all of them at the same time. NumPy does this effectively through "vectorization". In the following code, I have used this idea. Instead of passing one lattice-temperature pair, I have passed lists of lattice-temperature pairs. Now, in each step, I can calculate $\Delta E$, whether the state was accepted, and flipping the spin if the state was accepted, for all of the lattices in one go. The resulting speedup is huge, I estimate about $10$ times faster.

We have to keep $N$ fixed, otherwise the array indexing would get messed up when calculating $\Delta E$.

We can make further optimisations. There is a $2$ in the energy difference formula and we also defined $k=2$. These can be cancelled to remove two multiplications. Also, division is much slower than multiplication, so we can calculate $\frac{1}{T}$ beforehand and multiply by it instead of dividing in every step.

In [3]:
def mc_step(lattices,inv_Ts,N,i,j):
    #Calculate del_E
    del_Es = lattices[:,i,j]*(lattices[:,i-1,j]+lattices[:,i+1-N,j]+lattices[:,i,j-1]+lattices[:,i,j+1-N])
    #Check if state is accepted
    accepts = rng.random(inv_Ts.shape[0]) < np.exp(-del_Es*inv_Ts)
    #Flip if accepted
    #Accept value is True or False, so convert to 0 and 1, and map 0 -> 1 (no flip) and 1 -> -1 (flip)
    lattices[:,i,j] *= 1-2*accepts.astype(np.int8)

In [4]:
def mc_timestep(lattices,inv_Ts,N):
    for i in range(N):
        for j in range(N):
            mc_step(lattices,inv_Ts,N,i,j)

<h2>Simulating and gathering snapshots:</h2>

The following cells define the required elements of the simulation and then we run it. We take an $N \times N$ sized lattice at temperature $T$. For the initial state, we'll consider all spins to point up. First it is allowed to equilibriate for ```t_equilibrium``` timesteps, then ```snapshot_count``` snapshots are gathered every ```snapshot_interval``` timesteps.

Then the snapshots are stored in CSV format for further analysis. This way, we have to run the simulation just once, and later we can use the stored data anytime.

In [5]:
def simulate(N,Ts):
    #Create lattices
    lattices = np.ones(shape=(Ts.shape[0],N,N),dtype=np.int8)
    inv_Ts = 1/Ts

    #Allow them to equilibriate
    for t in trange(t_equilibrium,desc='Equilibriation'):
        mc_timestep(lattices,inv_Ts,N)

    #Collect snapshots
    snapshots = np.empty((snapshot_count,*lattices.shape),dtype=np.int8) #This stores the snapshots
    for i in trange(snapshot_count,desc='Collecting snapshots'):
        for t in range(snapshot_interval):
            mc_timestep(lattices,inv_Ts,N)
        snapshots[i] = lattices

    #Create data directory
    pathlib.Path(f'./data/{N}/').mkdir(parents=True,exist_ok=True)

    #Store snapshots as CSV
    for i in range(Ts.shape[0]):
      #Get the snapshots for fixed T
      snapshots_T = snapshots[:,i,:,:]
      #Flatten the 2D lattice into a 1D array for storing in CSV format
      snapshots_T = snapshots_T.reshape(snapshots_T.shape[0],-1)
      #Save as CSV
      np.savetxt(f'./data/{N}/{Ts[i]:.1f}.csv',snapshots_T,fmt='%d',delimiter=',')

In [6]:
t_equilibrium = 10000
snapshot_interval = 100
snapshot_count = 1000
N = 16
T_list = np.linspace(0.2,4,20)

In [ ]:
print(f'Processing N = {N}...')
simulate(N,T_list)

Processing N = 16...


Equilibriation:   0%|          | 0/10000 [00:00<?, ?it/s]